# Prompts Detailed

---

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLM

In [2]:
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt4",
    temperature=0,
)

## Prompts

`ChatPromptTemplate` is used to create a structured conversation with the AI model, making it easier to manage the flow and content of the conversation.

In [3]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

Inception is a 2010 science fiction action film written, directed, and produced by Christopher Nolan. The film stars Leonardo DiCaprio as Dom Cobb, a professional thief who specializes in stealing information from people's minds by infiltrating their dreams. The ensemble cast also includes Joseph Gordon-Levitt, Ellen Page, Tom Hardy, Ken Watanabe, Cillian Murphy, Marion Cotillard, and Michael Caine.

The story follows Cobb and his team as they attempt to plant an idea (inception) into the mind of a corporate heir, Robert Fischer (Cillian Murphy), by navigating through multiple layers of dreams. The film explores themes of reality, dreams, and the subconscious mind.

Inception was a critical and commercial success, grossing over $829 million worldwide. It received numerous awards and nominations, including eight Academy Award nominations, and won four Oscars for Best Cinematography, Best Sound Editing, Best Sound Mixing, and Best Visual Effects. The film is widely regarded for its innov

## Summarization chain example
In the case of a summarization task, we will leverage LangChain's in-built prompt using `load_summarize_chain`.

In [13]:
# Import necessary modules
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

# Load the summarization chain
summarize_chain = load_summarize_chain(chat)
summarize_chain.llm_chain.prompt

PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:', template_format='f-string', validate_template=True)

In [14]:
# Load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path="../../data/WritingArticleSummary.pdf")
document = document_loader.load()

# Summarize the document
summary = summarize_chain(document)
print(summary["output_text"])

An article summary is a short, focused paper about one scholarly article, informed by critical reading. It helps develop essential skills in critical reading, summarizing, and clear, organized writing. For argumentative articles, the summary identifies, explains, and analyzes the thesis and supporting arguments; for empirical articles, it identifies, explains, and analyzes the research questions, methods, and findings. The goal of an article summary is to write about the article, not about the actual topic of the article.


## QA chain example
In the case of a question-answering chain, LangChain can be used to manage prompts for extracting relevant information from documents. Below, we will define a custom prompt using LangChain. We can also use in-built prompt.

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    template="Question: {question}\nAnswer:", input_variables=["question"]
)

chain = LLMChain(llm=chat, prompt=prompt)
chain.run("what is the meaning of life?")

'The meaning of life is a philosophical question and has been debated for centuries. It generally refers to the purpose or significance of human existence. Different people, cultures, and belief systems have various interpretations of the meaning of life, ranging from religious or spiritual beliefs to personal values and goals. Ultimately, the meaning of life is subjective and can vary from person to person.'

## Chain Prompting
Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt. Usecase:
- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

In [5]:
from langchain import PromptTemplate, LLMChain

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=chat, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=chat, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: Albert Einstein's theory of general relativity, proposed in 1915, is a fundamental theory in physics that describes the force of gravity and its effects on the spacetime fabric. It states that massive objects, like planets and stars, cause a curvature in spacetime, which in turn influences the motion of other objects in their vicinity. This curvature of spacetime is experienced as gravity. The theory revolutionized our understanding of gravity, replacing Newton's classical theory, and has been confirmed through various experiments and observations, such as the bending of light around massive objects and the detection of gravitational waves.


## Chain of Thought prompting
Chain of Thought (CoT) prompting/reasoning refers to the strategy of asking the model to reason about a problem in steps before providing the final answer. For more details refer [this notebook](../build-systems-with-llm/4.chain-of-thought-reasoning.ipynb).

## Save and load prompts

In [12]:
template = """Answer the question based on the context below. If the \
question cannot be answered using the information provided, answer \
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum \
mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(input_variables=["query"], template=template)
prompt_template.template

'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".\nContext: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.\n...\nQuestion: {query}\nAnswer: '

In [ ]:
# Save prompt to a file
prompt_location = "./saved_prompts/"

prompt_template.save(f"{prompt_location}awesome_prompt.json")

In [11]:
# Load prompt from a file
from langchain.prompts import load_prompt

loaded_prompt = load_prompt(f"{prompt_location}awesome_prompt.json")
loaded_prompt.template

'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".\nContext: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.\n...\nQuestion: {query}\nAnswer: '

## Few shot prompting

### 1. No selector
Include all examples in the prompt.

In [4]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(
    llm=chat, prompt=PromptTemplate(template=formatted_prompt, input_variables=[])
)

# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain.run({})

print("Color: purple")
print("Emotion:", response)

Color: purple
Emotion: royalty


### 2. `LengthBasedExampleSelector`
It allow you to control the number of examples included based on query length. This helps in optimizing token usage and managing the balance between the number of examples and prompt size.

In [13]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!",
    },
    {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!",
    },
    {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems.",
    },
    {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'.",
    },
    {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite.",
    },
    {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes.",
    },
    {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'",
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump.",
    },
    {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies.",
    },
    {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world.",
    },
    {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom.",
    },
    {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette.",
    },
    {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa.",
    },
]

Instead of utilizing the examples list of dictionaries directly, we implement a `LengthBasedExampleSelector` like this:

In [15]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_formatter_template = """
query: {query}
answer: {answer}\n
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_formatter_template,
)

example_selector = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt, max_length=100
)

By employing the `LengthBasedExampleSelector`, the code dynamically selects and includes examples based on their length, ensuring that the final prompt stays within the desired token limit. The selector is employed to initialize a `dynamic_prompt_template`:

In [42]:
prefix = """You are an assistant that helps users by providing answers to \
their query. Here are some examples:\n"""

suffix = """\nquery: {query}\nanswer:
"""

dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n",
)

So, `the dynamic_prompt_template` utilizes the `example_selector` instead of a fixed list of examples. This allows the `FewShotPromptTemplate` to adjust the number of included examples **based on the length of the input query**. By doing so, it optimizes the use of the available context window and ensures that the language model receives an appropriate amount of contextual information.

In [43]:
# Create the LLMChain for the dynamic_prompt_template
chain = LLMChain(llm=chat, prompt=dynamic_prompt_template, verbose=True)

# Run the LLMChain with input_data
input_data = {"query": "Who invented the electric bulb?"}
response = chain.run(input_data)

print(response)



> Entering new  chain...
Prompt after formatting:
You are an assistant that helps users by providing answers to their query. Here are some examples:


query: How do you feel today?
answer: As an AI, I don't have feelings, but I've got jokes!



query: What is the speed of light?
answer: Fast enough to make a round trip around Earth 7.5 times in one second!



query: What is a quantum computer?
answer: A magical box that harnesses the power of subatomic particles to solve complex problems.



query: Who invented the telephone?
answer: Alexander Graham Bell, the original 'ringmaster'.



query: Who invented the electric bulb?
answer:


> Finished chain.
Thomas Edison, the bright spark who lit up our lives!


### 3. `SemanticSimilarityExampleSelector`
It selects examples based on their semantic resemblance to the input. 

Keep in mind that the `SemanticSimilarityExampleSelector` in the code below uses the Deep Lake vector store and `HuggingFaceEmbeddings` to measure semantic similarity. It stores the samples on the database in the cloud, and retrieves similar samples.

In [52]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# create Deep Lake dataset
# TODO: use your organization id here.  (by default, org id is your username)
my_activeloop_org_id = "iamrk04"
my_activeloop_dataset_name = "sample_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

# Embedding function
embeddings = HuggingFaceEmbeddings()

# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print("=" * 70)
print(similar_prompt.format(temperature="10°C"))  # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

# Add a new example to the SemanticSimilarityExampleSelector
print("=" * 70)
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(
    similar_prompt.format(temperature="40°C")
)  # Test with a new input after adding the example
print(
    similar_prompt.format(temperature="35°C")
)  # Test with a new input after adding the example

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (23, 768)  float32   None   
    id        text      (23, 1)     str     None   
 metadata     json      (23, 1)     str     None   
   text       text      (23, 1)     str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (24, 768)  float32   None   
    id        text      (24, 1)     str     None   
 metadata     json      (24, 1)     str     None   
   text       text      (24, 1)     str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 35°C
Output:
